In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##Required imports for notebook

In [ ]:
import numpy
import tensorflow as tf
import os
import os.path
import random
import math
import time
from PIL import Image
import sys
import scipy.ndimage
import subprocess
import matplotlib.pyplot


##Model class definition

In [ ]:
BATCH_SIZE = 1
KERNEL_SIZE = 3

class Model:
	def _conv_layer(self, name, input_var, stride, in_channels, out_channels, options = {}):
		activation = options.get('activation', 'relu')
		dropout = options.get('dropout', None)
		padding = options.get('padding', 'SAME')
		batchnorm = options.get('batchnorm', True)
		transpose = options.get('transpose', False)

		with tf.variable_scope(name) as scope:
			if not transpose:
				filter_shape = [KERNEL_SIZE, KERNEL_SIZE, in_channels, out_channels]
			else:
				filter_shape = [KERNEL_SIZE, KERNEL_SIZE, out_channels, in_channels]
			kernel = tf.get_variable(
				'weights',
				shape=filter_shape,
				initializer=tf.truncated_normal_initializer(stddev=math.sqrt(2.0 / KERNEL_SIZE / KERNEL_SIZE / in_channels)),
				dtype=tf.float32
			)
			biases = tf.get_variable(
				'biases',
				shape=[out_channels],
				initializer=tf.constant_initializer(0.0),
				dtype=tf.float32
			)
			if not transpose:
				output = tf.nn.bias_add(
					tf.nn.conv2d(
						input_var,
						kernel,
						[1, stride, stride, 1],
						padding=padding
					),
					biases
				)
			else:
				batch = tf.shape(input_var)[0]
				side = tf.shape(input_var)[1]
				output = tf.nn.bias_add(
					tf.nn.conv2d_transpose(
						input_var,
						kernel,
						[batch, side * stride, side * stride, out_channels],
						[1, stride, stride, 1],
						padding=padding
					),
					biases
				)
			if batchnorm:
				output = tf.contrib.layers.batch_norm(output, center=True, scale=True, is_training=self.is_training, decay=0.99)
			if dropout is not None:
				output = tf.nn.dropout(output, keep_prob=1-dropout)

			if activation == 'relu':
				return tf.nn.relu(output, name=scope.name)
			elif activation == 'sigmoid':
				return tf.nn.sigmoid(output, name=scope.name)
			elif activation == 'none':
				return output
			else:
				raise Exception('invalid activation {} specified'.format(activation))

	def _residual_layer(self, name, input_var, stride, in_channels, out_channels, options = {}):
		def bn_relu_conv_layer(name, input_var, stride, in_channels, out_channels, options = {}):
			with tf.variable_scope(name):
				padding = options.get('padding', 'SAME')
				transpose = options.get('transpose', False)
				first_block = options.get('first_block', False)

				if not first_block:
					input_var = tf.contrib.layers.batch_norm(input_var, center=True, scale=True, is_training=self.is_training, decay=0.99)
					input_var = tf.nn.relu(input_var)

				batch = tf.shape(input_var)[0]
				side = tf.shape(input_var)[1]

				if not transpose:
					filter_shape = [KERNEL_SIZE, KERNEL_SIZE, in_channels, out_channels]
				else:
					filter_shape = [KERNEL_SIZE, KERNEL_SIZE, out_channels, in_channels]
				kernel = tf.get_variable(
					'weights',
					shape=filter_shape,
					initializer=tf.truncated_normal_initializer(stddev=math.sqrt(2.0 / KERNEL_SIZE / KERNEL_SIZE / in_channels)),
					dtype=tf.float32
				)
				biases = tf.get_variable(
					'biases',
					shape=[out_channels],
					initializer=tf.constant_initializer(0.0),
					dtype=tf.float32
				)
				if not transpose:
					conv_output = tf.nn.bias_add(
						tf.nn.conv2d(
							input_var,
							kernel,
							[1, stride, stride, 1],
							padding=padding
						),
						biases
					)
				else:
					conv_output = tf.nn.bias_add(
						tf.nn.conv2d_transpose(
							input_var,
							kernel,
							[batch, side * stride, side * stride, out_channels],
							[1, stride, stride, 1],
							padding=padding
						),
						biases
					)

				return conv_output


		padding = options.get('padding', 'SAME')
		transpose = options.get('transpose', False)
		first_block = options.get('first_block', False)

		with tf.variable_scope(name):
			conv1 = bn_relu_conv_layer('conv1', input_var, stride, in_channels, out_channels, {'padding': padding, 'transpose': transpose, 'first_block': first_block})
			conv2 = bn_relu_conv_layer('conv2', input_var, 1, out_channels, out_channels, {'padding': padding})

			if stride == 1:
				output = input_var + conv2
			elif not transpose:
				fix_input = tf.nn.avg_pool(input_var, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
				if out_channels > in_channels:
					fix_input = tf.pad(fix_input, [[0, 0], [0, 0], [0, 0], [in_channels // 2, in_channels // 2]])
				output = fix_input + conv2
			else:
				fix_input = tf.image.resize_images(input_var, [side * stride, side * stride], tf.image.ResizeMethod.NEAREST_NEIGHBOR)
				output = fix_input + conv2

		return output

	def _fc_layer(self, name, input_var, input_size, output_size, options = {}):
		activation = options.get('activation', 'relu')
		dropout = options.get('dropout', None)
		batchnorm = options.get('batchnorm', True)

		with tf.variable_scope(name) as scope:
			weights = tf.get_variable(
				'weights',
				shape=[input_size, output_size],
				initializer=tf.truncated_normal_initializer(stddev=math.sqrt(2.0 / input_size)),
				dtype=tf.float32
			)
			biases = tf.get_variable(
				'biases',
				shape=[output_size],
				initializer=tf.constant_initializer(0.0),
				dtype=tf.float32
			)
			output = tf.matmul(input_var, weights) + biases
			if batchnorm:
				output = tf.contrib.layers.batch_norm(output, center=True, scale=True, is_training=self.is_training, decay=0.99)
			if dropout is not None:
				output = tf.nn.dropout(output, keep_prob=1-dropout)

			if activation == 'relu':
				return tf.nn.relu(output, name=scope.name)
			elif activation == 'sigmoid':
				return tf.nn.sigmoid(output, name=scope.name)
			elif activation == 'none':
				return output
			else:
				raise Exception('invalid activation {} specified'.format(activation))

	def __init__(self, mode=0, big=False):
		tf.reset_default_graph()

		self.is_training = tf.placeholder(tf.bool)
		if big:
			self.inputs = tf.placeholder(tf.float32, [None, 1024, 1024, 3])
			self.targets = tf.placeholder(tf.float32, [None, 1024, 1024, 1])
		else:
			self.inputs = tf.placeholder(tf.float32, [None, 256, 256, 3])
			self.targets = tf.placeholder(tf.float32, [None, 256, 256, 1])
		self.learning_rate = tf.placeholder(tf.float32)

		self.layer1 = self._conv_layer('layer1', self.inputs, 1, 3, 16, {'batchnorm': False}) # -> 1024x1024
		self.layer2 = self._residual_layer('layer2', self.layer1, 1, 16, 16, {'first_block': True}) # -> 1024x1024
		self.layer3 = self._residual_layer('layer3', self.layer2, 1, 16, 16) # -> 1024x1024
		self.layer4 = self._residual_layer('layer4', self.layer3, 1, 16, 16) # -> 1024x1024
		self.layer5 = self._residual_layer('layer5', self.layer4, 1, 16, 16) # -> 1024x1024
		self.layer6 = self._residual_layer('layer6', self.layer5, 1, 16, 16) # -> 1024x1024
		self.layer7 = self._residual_layer('layer7', self.layer6, 1, 16, 16) # -> 1024x1024

		self.layer8 = self._conv_layer('layer8', self.layer7, 2, 16, 32) # -> 512x512
		self.layer9 = self._residual_layer('layer9', self.layer8, 1, 32, 32) # -> 512x512
		self.layer10 = self._residual_layer('layer10', self.layer9, 1, 32, 32) # -> 512x512
		self.layer11 = self._residual_layer('layer11', self.layer10, 1, 32, 32) # -> 512x512
		self.layer12 = self._residual_layer('layer12', self.layer11, 1, 32, 32) # -> 512x512
		self.layer13 = self._residual_layer('layer13', self.layer12, 1, 32, 32) # -> 512x512
		self.layer14 = self._residual_layer('layer14', self.layer13, 1, 32, 32) # -> 512x512

		self.layer15 = self._conv_layer('layer15', self.layer14, 2, 32, 64) # -> 256x256
		self.layer16 = self._residual_layer('layer16', self.layer15, 1, 64, 64) # -> 256x256
		self.layer17 = self._residual_layer('layer17', self.layer16, 1, 64, 64) # -> 256x256
		self.layer18 = self._residual_layer('layer18', self.layer17, 1, 64, 64) # -> 256x256
		self.layer19 = self._residual_layer('layer19', self.layer18, 1, 64, 64) # -> 256x256
		self.layer20 = self._residual_layer('layer20', self.layer19, 1, 64, 64) # -> 256x256
		self.layer21 = self._residual_layer('layer21', self.layer20, 1, 64, 64) # -> 256x256

		self.layer22 = self._conv_layer('layer22', self.layer21, 2, 64, 128) # -> 128x128
		self.layer23 = self._residual_layer('layer23', self.layer22, 1, 128, 128) # -> 128x128
		self.layer24 = self._residual_layer('layer24', self.layer23, 1, 128, 128) # -> 128x128
		self.layer25 = self._residual_layer('layer25', self.layer24, 1, 128, 128) # -> 128x128
		self.layer26 = self._residual_layer('layer26', self.layer25, 1, 128, 128) # -> 128x128
		self.layer27 = self._residual_layer('layer27', self.layer26, 1, 128, 128) # -> 128x128
		self.layer28 = self._residual_layer('layer28', self.layer27, 1, 128, 128) # -> 128x128

		self.decoder1_inputs = tf.concat([self.layer28, self.layer22], axis=3)
		self.decoder1 = self._conv_layer('decoder1', self.decoder1_inputs, 2, 2*128, 64, {'transpose': True}) # -> 256x256
		self.decoder2_inputs = tf.concat([self.decoder1, self.layer15, self.layer21], axis=3)
		self.decoder2 = self._conv_layer('decoder2', self.decoder2_inputs, 2, 3*64, 32, {'transpose': True}) # -> 512x512
		self.decoder3_inputs = tf.concat([self.decoder2, self.layer8, self.layer14], axis=3)
		self.decoder3 = self._conv_layer('decoder3', self.decoder3_inputs, 2, 3*32, 16, {'transpose': True}) # -> 1024x1024

		self.initial_outputs = self._conv_layer('initial_outputs', self.decoder3, 1, 16, 2, {'activation': 'none', 'batchnorm': False}) # -> 1024x1024
		self.pre_outputs = tf.nn.softmax(self.initial_outputs)
		self.outputs = self.pre_outputs[:, :, :, 0]

		EPSILON = 1.0
		self.loss_numerator1 = tf.reduce_sum(self.pre_outputs[:, :, :, 0:1] * self.targets) + EPSILON
		self.loss_denominator1 = tf.reduce_sum(self.pre_outputs[:, :, :, 0:1] + self.targets - self.pre_outputs[:, :, :, 0:1] * self.targets) + EPSILON
		self.loss_numerator2 = tf.reduce_sum(self.pre_outputs[:, :, :, 1:2] * (1 - self.targets)) + EPSILON
		self.loss_denominator2 = tf.reduce_sum(self.pre_outputs[:, :, :, 1:2] + (1 - self.targets) - self.pre_outputs[:, :, :, 1:2] * (1 - self.targets)) + EPSILON
		if mode == 0:
			self.loss = -(self.loss_numerator1 / self.loss_denominator1 + self.loss_numerator2 / self.loss_denominator2 / 13)
		elif mode == 1:
			self.loss = -(self.loss_numerator1 / self.loss_denominator1 + self.loss_numerator2 / self.loss_denominator2)
		elif mode == 2:
			self.loss = -(self.loss_numerator1 / self.loss_denominator1)
		elif mode == 3:
			self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat([self.targets, 1 - self.targets], axis=3), logits=self.initial_outputs))
		elif mode == 4:
			self.loss = -(self.loss_numerator1 / self.loss_denominator1 + self.loss_numerator2 / self.loss_denominator2 / 2)

		with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
			self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)

		self.init_op = tf.initialize_all_variables()
		self.saver = tf.train.Saver(max_to_keep=None)


##Load dataset functions

In [ ]:
import numpy
import os
import os.path
import random
import scipy.ndimage
import scipy.misc
import multiprocessing
import time
import matplotlib.pyplot

SIZE = 256

def load_tile(sat_fname, osm_fname):
    #region = sat_fname.split('/')[-1].split('_sat')[0]
#     sat =  matplotlib.pyplot.imread(sat_fname)[0:1024, 0:1024, 0:3]
#     osm =  matplotlib.pyplot.imread(osm_fname)[0:1024, 0:1024].reshape(1024, 1024, 1)#return (region, sat, osm)
    #return (sat,osm)
    return (sat_fname,osm_fname)

def load_tiles(sat_path, osm_path):
	#files = [fname.split('_sat.png')[0] for fname in os.listdir(sat_path) if '_sat.png' in fname]
    files = [fname for fname in os.listdir(sat_path)]
# 	test_regions = ['boston', 'new york', 'chicago', 'la', 'toronto', 'denver', 'kansas city', 'san diego', 'pittsburgh', 'montreal', 'vancouver', 'tokyo', 'saltlakecity', 'paris', 'amsterdam']
# 	if traintest == 'train':
# 		files = [fname for fname in files if fname.split('/')[-1].split('_')[0] not in test_regions]
# 	elif traintest == 'test':
# 		files = [fname for fname in files if fname.split('/')[-1].split('_')[0] in test_regions]
# 	else:
# 		raise Exception('bad traintest {}'.format(traintest))

    tiles = []
    for i, fname in enumerate(files):
    	if i % 10 == 0:
    		print '{}/{}'.format(i, len(files))
    	sat_fname = '{}/{}'.format(sat_path, fname)
    	osm_fname = '{}/{}'.format(osm_path, fname)
    	tiles.append(load_tile(sat_fname, osm_fname))
    return tiles

def load_example(tile):
# 	sat, osm = tile
    sat_fname, osm_fname = tile
    sat =  matplotlib.pyplot.imread(sat_fname)[0:1024, 0:1024, 0:3]
    osm =  matplotlib.pyplot.imread(osm_fname)[0:1024, 0:1024].reshape(1024, 1024, 1)#return (region, sat, osm)
    i = random.randint(0, 1024 - 256)
    j = random.randint(0, 1024 - 256)
    example_sat = sat[i:i+256, j:j+256, :].astype('float32') / 255.0
    example_osm = osm[i:i+256, j:j+256, :].astype('float32') / 255.0
    return example_sat, example_osm

def load_all_examples(tile):
    sat_fname, osm_fname = tile
    #   sat, osm = tile
    sat =  matplotlib.pyplot.imread(sat_fname)[0:1024, 0:1024, 0:3]
    osm =  matplotlib.pyplot.imread(osm_fname)[0:1024, 0:1024].reshape(1024, 1024, 1)#return (region, sat, osm)
    examples = []
    for i in xrange(0, 1024, SIZE):
    	for j in xrange(0, 1024, SIZE):
    		example_sat = sat[i:i+SIZE, j:j+SIZE, :].astype('float32') / 255.0
    		example_osm = osm[i:i+SIZE, j:j+SIZE, :].astype('float32') / 255.0
    		examples.append((example_sat, example_osm))
    return examples


##Train segmentation

In [ ]:
sat_path = "/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/sat"
osm_path = "/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map"
model_path = "/content/gdrive/My Drive/model"

print 'loading train tiles'
train_tiles = load_tiles(sat_path, osm_path)

random.shuffle(train_tiles)
val_tiles = train_tiles[0:4]
train_tiles = train_tiles[4:]

val_examples = []
for tile in val_tiles:
	val_examples.extend(load_all_examples(tile))

print 'using {} train tiles, {} val tiles, {} val examples'.format(len(train_tiles), len(val_tiles), len(val_examples))

latest_path = model_path + '/model_latest/model'
best_path = model_path + '/model_best/model'

m = Model(mode=0)
session = tf.Session()
session.run(m.init_op)

best_loss = None

def epoch_to_learning_rate(epoch):
	if epoch < 100:
		return 1e-3
	elif epoch < 200:
		return 1e-4
	elif epoch < 300:
		return 1e-5
	elif epoch < 400:
		return 1e-6

for epoch in xrange(5):
    start_time = time.time()
    random.shuffle(train_tiles)
    train_losses = []
    print "Inside epoch"
    for x in xrange(5):
        #print x
        for i in xrange(0, len(train_tiles), BATCH_SIZE):
            #print "i="+str(i)
            examples = [ load_example(tile) for tile in train_tiles[i:i+ BATCH_SIZE]]
            _, loss = session.run([m.optimizer, m.loss], feed_dict={
				  m.is_training: True,
				  m.inputs: [example[0] for example in examples],
				  m.targets: [example[1] for example in examples],
				  m.learning_rate: 1e-3,
			   })
        train_losses.append(loss)
    train_loss = numpy.mean(train_losses)
    train_time = time.time()

    val_losses = []
    print "Going for validation"
    for i in xrange(0, len(val_examples),  BATCH_SIZE):
        #print "i="+str(i)
        examples = val_examples[i:i+ BATCH_SIZE]
        loss = session.run(m.loss, feed_dict={m.is_training: False,m.inputs: [example[0] for example in examples],m.targets: [example[1] for example in examples],})
        val_losses.append(loss)
    val_loss = numpy.mean(val_losses)
    val_time = time.time()
    print 'iteration {}: train_time={}, val_time={}, train_loss={}, val_loss={}/{}'.format(epoch, int(train_time - start_time), int(val_time - train_time), train_loss, val_loss, best_loss)
    m.saver.save(session, latest_path)
    if best_loss is None or val_loss < best_loss:
		best_loss = val_loss
		m.saver.save(session, best_path)

			

loading train tiles
0/1108
10/1108
20/1108
30/1108
40/1108
50/1108
60/1108
70/1108
80/1108
90/1108
100/1108
110/1108
120/1108
130/1108
140/1108
150/1108
160/1108
170/1108
180/1108
190/1108
200/1108
210/1108
220/1108
230/1108
240/1108
250/1108
260/1108
270/1108
280/1108
290/1108
300/1108
310/1108
320/1108
330/1108
340/1108
350/1108
360/1108
370/1108
380/1108
390/1108
400/1108
410/1108
420/1108
430/1108
440/1108
450/1108
460/1108
470/1108
480/1108
490/1108
500/1108
510/1108
520/1108
530/1108
540/1108
550/1108
560/1108
570/1108
580/1108
590/1108
600/1108
610/1108
620/1108
630/1108
640/1108
650/1108
660/1108
670/1108
680/1108
690/1108
700/1108
710/1108
720/1108
730/1108
740/1108
750/1108
760/1108
770/1108
780/1108
790/1108
800/1108
810/1108
820/1108
830/1108
840/1108
850/1108
860/1108
870/1108
880/1108
890/1108
900/1108
910/1108
920/1108
930/1108
940/1108
950/1108
960/1108
970/1108
980/1108
990/1108
1000/1108
1010/1108
1020/1108
1030/1108
1040/1108
1050/1108
1060/1108
1070/1108
1080/1108
1

##Run inference on training region

In [ ]:
"""
mkdir /data/deeproadmapper_test_out/
mkdir /data/deeproadmapper_test_out/im
mkdir /data/deeproadmapper_test_out/graph
"""


sat_path = "/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/sat"
osm_path = "/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map"
model_path = "/content/gdrive/My Drive/model"
out_path = "/content/gdrive/My Drive/output/img"

!mkdir -p /content/output/img

print 'loading train tiles'
train_tiles =  load_tiles(sat_path, osm_path)
best_path = model_path + '/model_best'
m =  Model(mode=0, big=True)
session = tf.Session()
m.saver.restore(session, tf.train.latest_checkpoint(best_path))


tiles = train_tiles
out_dir = out_path
for i, tile in enumerate(tiles):
	print '{}/{}'.format(i, len(tiles))
	sat_fname, _ = tile
	sat = matplotlib.pyplot.imread(sat_fname)[0:1024, 0:1024, 0:3]
	output = session.run(m.outputs, feed_dict={
		m.is_training: False,
		m.inputs: [sat.astype('float32') / 255.0],
		})[0, :, :]
	Image.fromarray((output * 255).astype('uint8')).save('{}/{}'.format(out_dir, os.path.basename(sat_fname)))
# 	print i



loading train tiles
0/1108
10/1108
20/1108
30/1108
40/1108
50/1108
60/1108
70/1108
80/1108
90/1108
100/1108
110/1108
120/1108
130/1108
140/1108
150/1108
160/1108
170/1108
180/1108
190/1108
200/1108
210/1108
220/1108
230/1108
240/1108
250/1108
260/1108
270/1108
280/1108
290/1108
300/1108
310/1108
320/1108
330/1108
340/1108
350/1108
360/1108
370/1108
380/1108
390/1108
400/1108
410/1108
420/1108
430/1108
440/1108
450/1108
460/1108
470/1108
480/1108
490/1108
500/1108
510/1108
520/1108
530/1108
540/1108
550/1108
560/1108
570/1108
580/1108
590/1108
600/1108
610/1108
620/1108
630/1108
640/1108
650/1108
660/1108
670/1108
680/1108
690/1108
700/1108
710/1108
720/1108
730/1108
740/1108
750/1108
760/1108
770/1108
780/1108
790/1108
800/1108
810/1108
820/1108
830/1108
840/1108
850/1108
860/1108
870/1108
880/1108
890/1108
900/1108
910/1108
920/1108
930/1108
940/1108
950/1108
960/1108
970/1108
980/1108
990/1108
1000/1108
1010/1108
1020/1108
1030/1108
1040/1108
1050/1108
1060/1108
1070/1108
1080/1108
1

##**RDP algorithm**

The Ramer-Douglas-Peucker algorithm roughly ported from the pseudo-code provided
by http://en.wikipedia.org/wiki/Ramer-Douglas-Peucker_algorithm


In [ ]:

from math import sqrt

def distance(a, b):
    return  sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def point_line_distance(point, start, end):
    if (start == end):
        return distance(point, start)
    else:
        n = abs(
            (end[0] - start[0]) * (start[1] - point[1]) - (start[0] - point[0]) * (end[1] - start[1])
        )
        d = sqrt(
            (end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2
        )
        return n / d

def rdp(points, epsilon):
    """
    Reduces a series of points to a simplified version that loses detail, but
    maintains the general shape of the series.
    """
    dmax = 0.0
    index = 0
    for i in range(1, len(points) - 1):
        d = point_line_distance(points[i], points[0], points[-1])
        if d > dmax:
            index = i
            dmax = d
    if dmax >= epsilon:
        results = rdp(points[:index+1], epsilon)[:-1] + rdp(points[index:], epsilon)
    else:
        results = [points[0], points[-1]]
    return results



##Map graph creation

In [ ]:
import scipy.ndimage
import skimage.morphology
import os
from PIL import Image
import math
import numpy
from multiprocessing import Pool
import subprocess

in_fname_path = "/content/gdrive/My Drive/output/img"
threshold = 10
out_fname_path = "/content/gdrive/My Drive/output/graph"
list_of_files = [f for f in os.listdir(in_fname_path) if os.path.isfile(os.path.join(in_fname_path,f))]
for f in list_of_files:
    in_fname = os.path.join(in_fname_path,f)
    im = scipy.ndimage.imread(in_fname)
    im = numpy.swapaxes(im, 0, 1)
    im = im > threshold

    # apply morphological dilation and thinning
    selem = skimage.morphology.disk(2)
    im = skimage.morphology.binary_dilation(im, selem)
    im = skimage.morphology.thin(im)
    im = im.astype('uint8')

    # extract a graph by placing vertices every THRESHOLD pixels, and at all intersections
    vertices = []
    edges = set()
    def add_edge(src, dst):
        if (src, dst) in edges or (dst, src) in edges:
            return
        elif src == dst:
            return
        edges.add((src, dst))
    point_to_neighbors = {}
    q = []
    while True:
        if len(q) > 0:
            lastid, i, j = q.pop()
            path = [vertices[lastid], (i, j)]
            if im[i, j] == 0:
                continue
            point_to_neighbors[(i, j)].remove(lastid)
            if len(point_to_neighbors[(i, j)]) == 0:
                del point_to_neighbors[(i, j)]
        else:
            w = numpy.where(im > 0)
            if len(w[0]) == 0:
                break
            i, j = w[0][0], w[1][0]
            lastid = len(vertices)
            vertices.append((i, j))
            path = [(i, j)]

        while True:
            im[i, j] = 0
            neighbors = []
            for oi in [-1, 0, 1]:
                for oj in [-1, 0, 1]:
                    ni = i + oi
                    nj = j + oj
                    if ni >= 0 and ni < im.shape[0] and nj >= 0 and nj < im.shape[1] and im[ni, nj] > 0:
                        neighbors.append((ni, nj))
            if len(neighbors) == 1 and (i, j) not in point_to_neighbors:
                ni, nj = neighbors[0]
                path.append((ni, nj))
                i, j = ni, nj
            else:
                if len(path) > 1:
                    path =  rdp(path, 2)
                    if len(path) > 2:
                        for point in path[1:-1]:
                            curid = len(vertices)
                            vertices.append(point)
                            add_edge(lastid, curid)
                            lastid = curid
                    neighbor_count = len(neighbors) + len(point_to_neighbors.get((i, j), []))
                    if neighbor_count == 0 or neighbor_count >= 2:
                        curid = len(vertices)
                        vertices.append(path[-1])
                        add_edge(lastid, curid)
                        lastid = curid
                for ni, nj in neighbors:
                    if (ni, nj) not in point_to_neighbors:
                        point_to_neighbors[(ni, nj)] = set()
                    point_to_neighbors[(ni, nj)].add(lastid)
                    q.append((lastid, ni, nj))
                for neighborid in point_to_neighbors.get((i, j), []):
                    add_edge(neighborid, lastid)
                break
    out_fname = os.path.join(out_fname_path,os.path.splitext(f)[0]+'.graph')
    print "Writting to graph file " + os.path.splitext(f)[0]+'.graph'
    with open(out_fname, 'w') as f:
        for vertex in vertices:
            f.write('{} {}\n'.format(vertex[0], vertex[1]))
        f.write('\n')
        for edge in edges:
            f.write('{} {}\n'.format(edge[0], edge[1]))
            f.write('{} {}\n'.format(edge[1], edge[0]))


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  app.launch_new_instance()


Writting to graph file 11878795_15.graph
Writting to graph file 11878780_15.graph
Writting to graph file 11878765_15.graph
Writting to graph file 11878750_15.graph
Writting to graph file 11878735_15.graph
Writting to graph file 11878720_15.graph
Writting to graph file 11878705_15.graph
Writting to graph file 11878690_15.graph
Writting to graph file 11878675_15.graph
Writting to graph file 11878660_15.graph
Writting to graph file 11728855_15.graph
Writting to graph file 11728840_15.graph
Writting to graph file 11728810_15.graph
Writting to graph file 11728795_15.graph
Writting to graph file 11728780_15.graph
Writting to graph file 11728765_15.graph
Writting to graph file 11728750_15.graph
Writting to graph file 11728735_15.graph
Writting to graph file 11728720_15.graph
Writting to graph file 11728705_15.graph
Writting to graph file 11728690_15.graph
Writting to graph file 11728675_15.graph
Writting to graph file 11728660_15.graph
Writting to graph file 11578840_15.graph
Writting to grap

##Geometric shapes

In [ ]:
import math

class Point(object):
	def __init__(self, x, y):
		self.x = int(x)
		self.y = int(y)

	def distance(self, other):
		dx = self.x - other.x
		dy = self.y - other.y
		return math.sqrt(dx * dx + dy * dy)

	def sub(self, other):
		return Point(self.x - other.x, self.y - other.y)

	def add(self, other):
		return Point(self.x + other.x, self.y + other.y)

	def scale(self, f):
		return Point(self.x * f, self.y * f)

	def magnitude(self):
		return math.sqrt(self.x * self.x + self.y * self.y)

	def angle_to(self, other):
		if self.magnitude() == 0 or other.magnitude() == 0:
			return 0
		s = (self.x * other.x + self.y * other.y) / self.magnitude() / other.magnitude()
		if abs(s) > 1: s = s / abs(s)
		angle = math.acos(s)
		if angle > math.pi:
			return 2 * math.pi - angle
		else:
			return angle

	def signed_angle(self, other):
		return math.atan2(other.y, other.x) - math.atan2(self.y, self.x)

	def bounds(self):
		return Rectangle(self, self)

	def dot(self, point):
		return self.x * point.x + self.y * point.y

	def __repr__(self):
		return 'Point({}, {})'.format(self.x, self.y)

	def __eq__(self, other):
		return self.x == other.x and self.y == other.y

	def __ne__(self, other):
		return not self.__eq__(other)

	def __hash__(self):
		return hash((self.x, self.y))

class FPoint(object):
	def __init__(self, x, y):
		self.x = float(x)
		self.y = float(y)

	def distance(self, other):
		dx = self.x - other.x
		dy = self.y - other.y
		return math.sqrt(dx * dx + dy * dy)

	def sub(self, other):
		return FPoint(self.x - other.x, self.y - other.y)

	def add(self, other):
		return FPoint(self.x + other.x, self.y + other.y)

	def scale(self, f):
		return FPoint(self.x * f, self.y * f)

	def magnitude(self):
		return math.sqrt(self.x * self.x + self.y * self.y)

	def angle_to(self, other):
		if self.magnitude() == 0 or other.magnitude() == 0:
			return 0
		s = (self.x * other.x + self.y * other.y) / self.magnitude() / other.magnitude()
		if abs(s) > 1: s = s / abs(s)
		angle = math.acos(s)
		if angle > math.pi:
			return 2 * math.pi - angle
		else:
			return angle

	def signed_angle(self, other):
		return math.atan2(other.y, other.x) - math.atan2(self.y, self.x)

	def dot(self, point):
		return self.x * point.x + self.y * point.y

	def __repr__(self):
		return 'FPoint({}, {})'.format(self.x, self.y)

class Segment(object):
	def __init__(self, start, end):
		self.start = start
		self.end = end

	def length(self):
		return self.start.distance(self.end)

	def project_factor(self, point):
		l = self.length()
		if l == 0:
			return 0
		t = point.sub(self.start).dot(self.end.sub(self.start)) / l
		t = max(0, min(l, t))
		return t

	def project(self, point):
		t = self.project_factor(point)
		return self.point_at_factor(t)

	def point_at_factor(self, t):
		l = self.length()
		if l == 0:
			return self.start
		return self.start.add(self.end.sub(self.start).scale(t / l))

	def distance(self, point):
		p = self.project(point)
		return p.distance(point)

	def intersection(self, other):
		d1 = self.vector()
		d2 = other.vector()
		d12 = other.start.sub(self.start)

		den = d1.y * d2.x - d1.x * d2.y
		u1 = d1.x * d12.y - d1.y * d12.x
		u2 = d2.x * d12.y - d2.y * d12.x

		if den == 0:
			# collinear
			if u1 == 0 and u2 == 0:
				return self.start
			else:
				return None

		if float(u1) / den < 0 or float(u1) / den > 1 or float(u2) / den < 0 or float(u2) / den > 1:
			return None

		return self.point_at_factor(float(u2) / den * self.length())

	def vector(self):
		return self.end.sub(self.start)

	def bounds(self):
		return self.start.bounds().extend(self.end)

	def extend(self, amount):
		v = self.vector()
		v = v.scale(amount / v.magnitude())
		return Segment(
			self.start.sub(v),
			self.end.add(v)
		)

	def __repr__(self):
		return 'Segment({}, {})'.format(self.start, self.end)

class Rectangle(object):
	def __init__(self, start, end):
		self.start = start
		self.end = end

	def lengths(self):
		return Point(self.end.x - self.start.x, self.end.y - self.start.y)

	def clip(self, point):
		npoint = Point(point.x, point.y)
		if npoint.x < self.start.x:
			npoint.x = self.start.x
		elif npoint.x >= self.end.x:
			npoint.x = self.end.x - 1
		if npoint.y < self.start.y:
			npoint.y = self.start.y
		elif npoint.y >= self.end.y:
			npoint.y = self.end.y - 1
		return npoint

	def clip_rect(self, r):
		return Rectangle(self.clip(r.start), self.clip(r.end))

	def add_tol(self, tol):
		return Rectangle(
			self.start.sub(Point(tol, tol)),
			self.end.add(Point(tol, tol))
		)

	def contains(self, point):
		return point.x >= self.start.x and point.x < self.end.x and point.y >= self.start.y and point.y < self.end.y

	def extend(self, point):
		return Rectangle(
			Point(min(self.start.x, point.x), min(self.start.y, point.y)),
			Point(max(self.end.x, point.x), max(self.end.y, point.y))
		)

	def intersects(self, other):
		return self.end.y >= other.start.y and other.end.y >= self.start.y and self.end.x >= other.start.x and other.end.x >= self.start.x

def draw_line(start, end, lengths):
	# followX indicates whether to move along x or y coordinates
	followX = abs(end.y - start.y) <= abs(end.x - start.x)
	if followX:
		x0 = start.x
		x1 = end.x
		y0 = start.y
		y1 = end.y
	else:
		x0 = start.y
		x1 = end.y
		y0 = start.x
		y1 = end.x

	delta = Point(abs(x1 - x0), abs(y1 - y0))
	current_error = 0

	if x0 < x1:
		xstep = 1
	else:
		xstep = -1

	if y0 < y1:
		ystep = 1
	else:
		ystep = -1

	points = []
	def add_point(p):
		if p.x >= 0 and p.x < lengths.x and p.y >= 0 and p.y < lengths.y:
			points.append(p)

	x = x0
	y = y0

	while x != x1 + xstep:
		if followX:
			add_point(Point(x, y))
		else:
			add_point(Point(y, x))

		x += xstep
		current_error += delta.y
		if current_error >= delta.x:
			y += ystep
			current_error -= delta.x

	return points

def vector_from_angle(angle, length):
	return Point(math.cos(angle) * length, math.sin(angle) * length)


##Graph helper functions

In [ ]:
!apt install libspatialindex-dev
!pip install Rtree

import rtree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 1s (454 kB/s)
Selecting previously unselected package libspatialindex

In [ ]:


class Vertex(object):
	def __init__(self, id, point):
		self.id = id
		self.point = point
		self.in_edges = []
		self.out_edges = []

	def _neighbors(self):
		n = {}
		for edge in self.in_edges:
			n[edge.src] = edge
		for edge in self.out_edges:
			n[edge.dst] = edge
		return n

	def neighbors(self):
		return self._neighbors().keys()
  
##########################################################################################



class Edge(object):
	def __init__(self, id, src, dst):
		self.id = id
		self.src = src
		self.dst = dst

	def bounds(self):
		return self.src.point.bounds().extend(self.dst.point)

	def segment(self):
		return Segment(self.src.point, self.dst.point)

	def closest_pos(self, point):
		p = self.segment().project(point)
		return EdgePos(self, p.distance(self.src.point))

	def is_opposite(self, edge):
		return edge.src == self.dst and edge.dst == self.src

	def get_opposite_edge(self):
		for edge in self.dst.out_edges:
			if self.is_opposite(edge):
				return edge
		return None

	def is_adjacent(self, edge):
		return edge.src == self.src or edge.src == self.dst or edge.dst == self.src or edge.dst == self.dst
  
##########################################################################################

class EdgePos(object):
	def __init__(self, edge, distance):
		self.edge = edge
		self.distance = distance

	def point(self):
		segment = self.edge.segment()
		vector = segment.vector()
		if vector.magnitude() < 1:
			return segment.start
		else:
			return segment.start.add(vector.scale(self.distance / vector.magnitude()))

	def reverse(self):
		return EdgePos(self.edge.get_opposite_edge(), self.edge.segment().length() - self.distance)

##########################################################################################
class Index(object):
	def __init__(self, graph, index):
		self.graph = graph
		self.index = index

	def search(self, rect):
		edge_ids = self.index.intersection((rect.start.x, rect.start.y, rect.end.x, rect.end.y))
		return [self.graph.edges[edge_id] for edge_id in edge_ids]

	def subgraph(self, rect):
		ng = Graph()
		vertex_map = {}
		for edge in self.search(rect):
			for vertex in [edge.src, edge.dst]:
				if vertex not in vertex_map:
					vertex_map[vertex] = ng.add_vertex(vertex.point)
			ng.add_edge(vertex_map[edge.src], vertex_map[edge.dst])
		return ng

  
##########################################################################################  
  
class Graph(object):
	def __init__(self):
		self.vertices = []
		self.edges = []

	def add_vertex(self, point):
		vertex = Vertex(len(self.vertices), point)
		self.vertices.append(vertex)
		return vertex

	def add_edge(self, src, dst):
		if src == dst:
			raise Exception('cannot add edge between same vertex')
		edge = Edge(len(self.edges), src, dst)
		self.edges.append(edge)
		src.out_edges.append(edge)
		dst.in_edges.append(edge)
		return edge

	def add_bidirectional_edge(self, src, dst):
		return (
			self.add_edge(src, dst),
			self.add_edge(dst, src),
		)

	def edgeIndex(self):
		rt = rtree.index.Index()
		for edge in self.edges:
			bounds = edge.bounds()
			rt.insert(edge.id, (bounds.start.x, bounds.start.y, bounds.end.x, bounds.end.y))
		return Index(self, rt)

	def bounds(self):
		r = None
		for vertex in self.vertices:
			if r is None:
				r = Rectangle(vertex.point, vertex.point)
			else:
				r = r.extend(vertex.point)
		return r

	def save(self, fname):
		with open(fname, 'w') as f:
			for vertex in self.vertices:
				f.write("{} {}\n".format(vertex.point.x, vertex.point.y))
			f.write("\n")
			for edge in self.edges:
				f.write("{} {}\n".format(edge.src.id, edge.dst.id))

	def clone(self):
		other = Graph()
		for vertex in self.vertices:
			v = other.add_vertex(vertex.point)
			if hasattr(vertex, 'edge_pos'):
				v.edge_pos = vertex.edge_pos
		for edge in self.edges:
			e = other.add_edge(other.vertices[edge.src.id], other.vertices[edge.dst.id])
		return other

	def filter_edges(self, filter_edges):
		g = Graph()
		vertex_map = {}
		for edge in self.edges:
			if edge in filter_edges:
				continue
			for vertex in [edge.src, edge.dst]:
				if vertex not in vertex_map:
					vertex_map[vertex] = g.add_vertex(vertex.point)
			g.add_edge(vertex_map[edge.src], vertex_map[edge.dst])
		return g

def read_graph(fname, merge_duplicates=False):
	graph = Graph()
	with open(fname, 'r') as f:
		vertex_section = True
		vertices = {}
		next_vertex_id = 0
		seen_points = {}
		for line in f:
			parts = line.strip().split(' ')
			if vertex_section:
				if len(parts) >= 2:
					point = Point(float(parts[0]), float(parts[1]))
					if point in seen_points and merge_duplicates:
						print 'merging duplicate vertex at {}'.format(point)
						vertices[next_vertex_id] = seen_points[point]
					else:
						vertex = graph.add_vertex(point)
						vertices[next_vertex_id] = vertex
						seen_points[point] = vertex
					next_vertex_id += 1
				else:
					vertex_section = False
			elif len(parts) >= 2:
				src = vertices[int(parts[0])]
				dst = vertices[int(parts[1])]
				if src == dst and merge_duplicates:
					print 'ignoring self edge at {}'.format(src.point)
					continue
				graph.add_edge(src, dst)
	return graph

def dijkstra_helper(src, stop_at=None, max_distance=None):
	distances = {}
	prev = {}
	remaining = set()
	seen = set()

	distances[src.id] = 0
	remaining.add(src)
	seen.add(src)

	while len(remaining) > 0:
		closestNode = None
		closestDistance = None
		for vertex in remaining:
			if closestNode is None or distances[vertex.id] < closestDistance:
				closestNode = vertex
				closestDistance = distances[vertex.id]
		remaining.remove(closestNode)

		if stop_at is not None and stop_at == closestNode:
			break
		elif closestDistance > max_distance:
			break

		for other, edge in closestNode._neighbors().items():
			if other not in seen:
				seen.add(other)
				remaining.add(other)
				distances[other.id] = float('inf')
				prev[other.id] = None

			if other in remaining:
				if hasattr(edge, 'cost'):
					d = closestDistance + edge.cost
				else:
					d = closestDistance + closestNode.point.distance(other.point)
				if d < distances[other.id]:
					distances[other.id] = d
					prev[other.id] = (closestNode, edge)

	return distances, prev

def shortest_distances_from_source(src, max_distance=None):
	distances, _ = dijkstra_helper(src, max_distance=max_distance)
	return distances

def shortest_path(src, dst, max_distance=None):
	_, prev = dijkstra_helper(src, stop_at=dst, max_distance=max_distance)
	if dst.id not in prev:
		return None, None
	vertex_path = [dst]
	edge_path = []
	cur_node = dst
	while cur_node != src:
		cur_node, edge = prev[cur_node.id]
		vertex_path.append(cur_node)
		edge_path.append(edge)
	vertex_path.reverse()
	edge_path.reverse()
	return vertex_path, edge_path

# searches for the closest edge position to the specified point
# if src is specified, only looks for edges reachable from the src edge position within remaining units
def closest_reachable_edge(point, index, explored_node_pairs=None, remaining = 50, src = None, distance_threshold = 50):
	closest_edge_pos = None
	closest_edge_pos_path = None
	smallest_distance = None

	# if src is None, get candidate edges from index
	# otherwise, get candidate edges by following graph
	if src is None:
		candidates = [(edge, None) for edge in index.search(point.bounds().add_tol(distance_threshold))]
	else:
		candidates = [(src.edge, None)]
		cur_explored_node_pairs = set()
		cur_explored_node_pairs.add((src.edge.src.id, src.edge.dst.id))
		def search_edge(path, edge, remaining):
			l = edge.segment().length()
			if remaining > l:
				search_vertex(path + [edge], edge.dst, remaining - l)
		def search_vertex(path, vertex, remaining):
			for edge in vertex.out_edges:
				if (edge.src.id, edge.dst.id) in cur_explored_node_pairs or (edge.dst.id, edge.src.id) in cur_explored_node_pairs:
					continue
				candidates.append((edge, path))
				cur_explored_node_pairs.add((edge.src.id, edge.dst.id))
				search_edge(path, edge, remaining)

		# search forwards
		l = src.edge.segment().length() - src.distance
		if remaining > l:
			search_vertex([], src.edge.dst, remaining - l)

		# search backwards
		reverse_edge_pos = src.reverse()
		l = reverse_edge_pos.edge.segment().length() - reverse_edge_pos.distance
		if remaining > l:
			search_vertex([], reverse_edge_pos.edge.dst, remaining - l)

	for edge, path in candidates:
		if explored_node_pairs is not None and ((edge.src.id, edge.dst.id) in explored_node_pairs or (edge.dst.id, edge.src.id) in explored_node_pairs) and (src is None or edge != src.edge):
			continue
		distance = edge.segment().distance(point)
		if distance > distance_threshold:
			continue

		if closest_edge_pos is None or distance < smallest_distance:
			closest_edge_pos = edge.closest_pos(point)
			closest_edge_pos_path = path
			smallest_distance = distance

	return closest_edge_pos, closest_edge_pos_path

def follow_graph(edge_pos, distance, explored_node_pairs=None):
	if explored_node_pairs:
		explored_node_pairs = set(explored_node_pairs)
	else:
		explored_node_pairs = set()
	explored_node_pairs.add((edge_pos.edge.src.id, edge_pos.edge.dst.id))
	positions = []

	def search_edge(edge, remaining):
		l = edge.segment().length()
		if remaining > l:
			search_vertex(edge.dst, remaining - l)
		else:
			pos = EdgePos(edge, remaining)
			positions.append(pos)

	def search_vertex(vertex, remaining):
		for edge in vertex.out_edges:
			if (edge.src.id, edge.dst.id) in explored_node_pairs or (edge.dst.id, edge.src.id) in explored_node_pairs:
				continue
			explored_node_pairs.add((edge.src.id, edge.dst.id))
			search_edge(edge, remaining)

	remaining = distance
	l = edge_pos.edge.segment().length() - edge_pos.distance
	if remaining > l:
		search_vertex(edge_pos.edge.dst, remaining - l)
	else:
		positions = [EdgePos(edge_pos.edge, edge_pos.distance + remaining)]

	return positions

class RoadSegment(object):
	def __init__(self, id):
		self.id = id
		self.edges = []

	def add_edge(self, edge, direction):
		if direction == 'forwards':
			self.edges.append(edge)
		elif direction == 'backwards':
			self.edges = [edge] + self.edges
		else:
			raise Exception('bad edge')

	def compute_edge_distances(self):
		l = 0
		self.edge_distances = {}
		for edge in self.edges:
			self.edge_distances[edge.id] = l
			l += edge.segment().length()

	def distance_to_edge(self, distance, return_idx=False):
		for i in xrange(len(self.edges)):
			edge = self.edges[i]
			distance -= edge.segment().length()
			if distance <= 0:
				if return_idx:
					return i
				else:
					return edge
		if return_idx:
			return len(self.edges) - 1
		else:
			return self.edges[-1]

	def src(self):
		return self.edges[0].src

	def dst(self):
		return self.edges[-1].dst

	def is_opposite(self, rs):
		return self.src() == rs.dst() and self.dst() == rs.src() and self.edges[0].is_opposite(rs.edges[-1])

	def in_rs(self, edge_to_rs):
		rs_set = {}
		for edge in self.src().in_edges:
			rs = edge_to_rs[edge.id]
			if rs.id != self.id and rs.id not in rs_set:
				rs_set[rs.id] = rs
		return rs_set.values()

	def out_rs(self, edge_to_rs):
		rs_set = {}
		for edge in self.dst().out_edges:
			rs = edge_to_rs[edge.id]
			if rs.id != self.id and rs.id not in rs_set:
				rs_set[rs.id] = rs
		return rs_set.values()

	def get_opposite_rs(self, edge_to_rs):
		for rs in self.out_rs(edge_to_rs):
			if self.is_opposite(rs):
				return rs
		return None

	def length(self):
		return sum([edge.segment().length() for edge in self.edges])

	def closest_pos(self, point):
		best_edge_pos = None
		best_distance = None
		for edge in self.edges:
			edge_pos = edge.closest_pos(point)
			distance = edge_pos.point().distance(point)
			if best_edge_pos is None or distance < best_distance:
				best_edge_pos = edge_pos
				best_distance = distance
		return best_edge_pos

	def point_at_factor(self, t):
		edge = self.distance_to_edge(t)
		return edge.segment().point_at_factor(t - self.edge_distances[edge.id])

def get_graph_road_segments(g):
	road_segments = []
	edge_to_rs = {}

	def search_from_edge(rs, edge, direction):
		cur_edge = edge
		while True:
			if direction == 'forwards':
				vertex = cur_edge.dst
				edges = vertex.out_edges
			elif direction == 'backwards':
				vertex = cur_edge.src
				edges = vertex.in_edges

			edges = [next_edge for next_edge in edges if not next_edge.is_opposite(cur_edge)]

			if len(edges) != 1:
				# we have hit intersection vertex or a dead end
				return

			next_edge = edges[0]

			if next_edge.id in edge_to_rs:
				# this should only happen when we run in a segment that is actually a loop
				# although really it shouldn't happen in that case either, since loops should start/end at an intersection
				# TODO: think about this more
				return

			rs.add_edge(next_edge, direction)
			edge_to_rs[next_edge.id] = rs
			cur_edge = next_edge

	for edge in g.edges:
		if edge.id in edge_to_rs:
			continue

		rs = RoadSegment(len(road_segments))
		rs.add_edge(edge, 'forwards')
		edge_to_rs[edge.id] = rs
		search_from_edge(rs, edge, 'forwards')
		search_from_edge(rs, edge, 'backwards')
		rs.compute_edge_distances()
		road_segments.append(rs)

	return road_segments, edge_to_rs

class GraphContainer(object):
	def __init__(self, g):
		self.graph = g
		self.edge_index = g.edgeIndex()
		self.road_segments, self.edge_to_rs = get_graph_road_segments(g)

def mapmatch(index, road_segments, edge_to_rs, points, segment_length):
	SIGMA = segment_length
	START_TOL = segment_length * 2.5
	MAX_TOL = segment_length * 4

	# probs keeps track of both the emission probability and the distance along road segment of the
	#  point closest to the previous path point
	# on the next iteration, we restrict to choosing points at higher distances along road segment
	probs = {}

	for edge in index.search(points[0].bounds().add_tol(START_TOL)):
		rs = edge_to_rs[edge.id]
		distance = edge.segment().distance(points[0])
		p = -0.5 * distance * distance / SIGMA / SIGMA
		if rs.id not in probs or p > probs[rs.id][0]:
			rs_distance = rs.edge_distances[edge.id] + edge.segment().project_factor(points[0])
			probs[rs.id] = (p, rs_distance)

	# given a road segment and a previous rs distance, returns next rs distance and the actual point distance
	def distance_to_rs(rs, clip_distance, point, vector):
		low_distance = clip_distance + segment_length / 2
		high_distance = clip_distance + segment_length * 2

		if high_distance < 0 or low_distance > rs.length():
			return None, None

		# get edges between low_distance and high_distance
		low_edge_idx = rs.distance_to_edge(low_distance, return_idx=True)
		high_edge_idx = rs.distance_to_edge(high_distance, return_idx=True)
		edges = rs.edges[low_edge_idx:high_edge_idx+1]

		best_distance = None
		best_rs_distance = None
		for edge in edges:
			rs_distance = rs.edge_distances[edge.id] + edge.segment().project_factor(point)
			rs_distance = numpy.clip(rs_distance, low_distance, high_distance)
			rs_distance = numpy.clip(rs_distance, rs.edge_distances[edge.id], rs.edge_distances[edge.id] + edge.segment().length())
			edge_distance = rs_distance - rs.edge_distances[edge.id]
			rs_point = edge.segment().point_at_factor(edge_distance)
			distance = rs_point.distance(point) + edge.segment().vector().angle_to(vector) * 12
			if best_distance is None or distance < best_distance:
				best_distance = distance
				best_rs_distance = rs_distance

		return best_distance, best_rs_distance

	backpointers = [{} for _ in xrange(len(points) - 1)]

	for i in xrange(len(points) - 1):
		next_probs = {}
		for prev_rs_id in probs:
			prev_p, prev_rs_distance = probs[prev_rs_id]
			rs = road_segments[prev_rs_id]
			remaining_rs_distance = rs.length() - prev_rs_distance
			rs_candidates = [(rs, prev_rs_distance)] + [(next_rs, -remaining_rs_distance) for next_rs in rs.out_rs(edge_to_rs)]
			for next_rs, clip_distance in rs_candidates:
				if next_rs.is_opposite(rs):
					continue
				distance, rs_distance = distance_to_rs(next_rs, clip_distance, points[i + 1], points[i + 1].sub(points[i]))
				if distance is None or distance > MAX_TOL:
					continue
				p = prev_p + (-0.5 * distance * distance / SIGMA / SIGMA)
				#if next_rs != rs:
				#	p += math.log(0.1)
				if next_rs.id not in next_probs or p > next_probs[next_rs.id][0]:
					next_probs[next_rs.id] = (p, rs_distance)
					backpointers[i][next_rs.id] = prev_rs_id
		probs = next_probs
		if len(probs) == 0:
			return None, None

	return probs, backpointers

def mm_best_rs(road_segments, probs, rs_blacklist=None):
	best_rs = None
	for rs_id in probs:
		if rs_blacklist is not None and rs_id in rs_blacklist:
			continue
		prob = probs[rs_id][0]
		if best_rs is None or prob > probs[best_rs.id][0]:
			best_rs = road_segments[rs_id]
	return best_rs

def mm_follow_backpointers(road_segments, rs_id, backpointers):
	rs_list = []
	for i in xrange(len(backpointers) - 1, -1, -1):
		rs_id = backpointers[i][rs_id]
		rs_list.append(road_segments[rs_id])
	rs_list.reverse()
	return rs_list

def get_nearby_vertices(vertex, n):
	nearby_vertices = set()
	def search(vertex, remaining):
		if vertex in nearby_vertices:
			return
		nearby_vertices.add(vertex)
		if remaining == 0:
			return
		for edge in vertex.in_edges:
			search(edge.src, remaining - 1)
			search(edge.dst, remaining - 1)
	search(vertex, n)
	return nearby_vertices

##Get connections

In [ ]:


import numpy
import random

MIN_GRAPH_DISTANCE = 166
MAX_STRAIGHT_DISTANCE = 83
RDP_EPSILON = 2

def get_connections(g, im, limit=None):
	edge_im = -numpy.ones(im.shape, dtype='int32')
	for edge in g.edges:
		for p in    draw_line(edge.src.point, edge.dst.point,    Point(edge_im.shape[0], edge_im.shape[1])):
			edge_im[p.x, p.y] = edge.id

	road_segments, _ =  get_graph_road_segments(g)
	random.shuffle(road_segments)
	best_rs = None
	seen_vertices = set()
	proposed_connections = []
	for rs in road_segments:
		for vertex, opp in [(rs.src(), rs.point_at_factor(10)), (rs.dst(), rs.point_at_factor(rs.length() - 10))]:
			if len(vertex.out_edges) >= 2 or vertex in seen_vertices:
				continue
			seen_vertices.add(vertex)

			vertex_distances = get_vertex_distances(vertex, MIN_GRAPH_DISTANCE)
			edge, path = get_shortest_path(im, vertex.point, opp, edge_im, g, vertex_distances)
			if edge is not None:
				proposed_connections.append({
					'src': vertex.id,
					'edge': edge.id,
					'pos': edge.closest_pos(path[-1]).distance,
					'path':  rdp([(p.x, p.y) for p in path], RDP_EPSILON),
				})
		if limit is not None and len(proposed_connections) >= limit:
			break

	return proposed_connections

def insert_connections(g, connections):
	split_edges = {} # map from edge to (split pos, new edge before pos, new edge after pos)
	for idx, connection in enumerate(connections):
		# figure out which current edge the connection intersects
		edge = g.edges[connection['edge']]
		path = [   Point(p[0], p[1]) for p in connection['path']]
		intersection_point = path[-1]
		while edge in split_edges:
			our_pos = edge.closest_pos(intersection_point).distance
			if our_pos < split_edges[edge]['pos']:
				edge = split_edges[edge]['before']
			else:
				edge = split_edges[edge]['after']

		# add path vertices
		prev_vertex = g.vertices[connection['src']]
		for point in path[1:]:
			vertex = g.add_vertex(point)
			edge1, edge2 = g.add_bidirectional_edge(prev_vertex, vertex)
			edge1.phantom = True
			edge1.connection_idx = idx
			edge2.phantom = True
			edge2.connection_idx = idx
			prev_vertex = vertex

		# split the edge
		new_vertex = prev_vertex
		for edge in [edge, edge.get_opposite_edge()]:
			split_pos = edge.closest_pos(intersection_point).distance
			split_edges[edge] = {
				'pos': split_pos,
				'before': g.add_edge(edge.src, new_vertex),
				'after': g.add_edge(new_vertex, edge.dst),
			}

	# remove extraneous edges
	filter_edges = set([edge for edge in split_edges.keys()])
	g = g.filter_edges(filter_edges)
	return g

def get_vertex_distances(src, max_distance):
	vertex_distances = {}

	seen_vertices = set()
	distances = {}
	distances[src] = 0
	while len(distances) > 0:
		closest_vertex = None
		closest_distance = None
		for vertex, distance in distances.items():
			if closest_vertex is None or distance < closest_distance:
				closest_vertex = vertex
				closest_distance = distance

		del distances[closest_vertex]
		vertex_distances[closest_vertex] = closest_distance
		seen_vertices.add(closest_vertex)
		if closest_distance > max_distance:
			break

		for edge in closest_vertex.out_edges:
			vertex = edge.dst
			if hasattr(edge, 'cost'):
				distance = closest_distance + edge.cost
			else:
				distance = closest_distance + edge.segment().length()
			if vertex not in seen_vertices and (vertex not in distances or distance < distances[vertex]):
				distances[vertex] = distance

	return vertex_distances

def get_shortest_path(im, src, opp, edge_im, g, vertex_distances):
	r = src.bounds().add_tol(MAX_STRAIGHT_DISTANCE)
	r =    Rectangle(   Point(0, 0),    Point(im.shape[0], im.shape[1])).clip_rect(r)
	seen_points = set()
	distances = {}
	prev = {}
	dst_edge = None
	dst_point = None

	distances[src] = 0
	while len(distances) > 0:
		closest_point = None
		closest_distance = None
		for point, distance in distances.items():
			if closest_point is None or distance < closest_distance:
				closest_point = point
				closest_distance = distance

		del distances[closest_point]
		seen_points.add(closest_point)
		if edge_im[closest_point.x, closest_point.y] >= 0:
			edge = g.edges[edge_im[closest_point.x, closest_point.y]]
			src_distance = vertex_distances.get(edge.src, MIN_GRAPH_DISTANCE)
			dst_distance = vertex_distances.get(edge.dst, MIN_GRAPH_DISTANCE)
			if src_distance + closest_point.distance(edge.src.point) >= MIN_GRAPH_DISTANCE and dst_distance + closest_point.distance(edge.dst.point) >= MIN_GRAPH_DISTANCE:
				dst_edge = edge
				dst_point = closest_point
				break

		for offset in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
			adj_point = closest_point.add(   Point(offset[0], offset[1]))
			if r.contains(adj_point) and adj_point not in seen_points and src.distance(adj_point) < opp.distance(adj_point):
				distance = closest_distance + 1 + (1 - im[adj_point.x, adj_point.y])
				if adj_point not in distances or distance < distances[adj_point]:
					distances[adj_point] = distance
					prev[adj_point] = closest_point

	if dst_edge is None:
		return None, None

	path = []
	point = dst_point
	while point != src:
		path.append(point)
		point = prev[point]
	path.append(src)
	path.reverse()

	return dst_edge, path


##Label connections

In [ ]:


import numpy
from PIL import Image

PHANTOM_BASE = 8
PHANTOM_WEIGHT = 3
MAX_GT_DISTANCE = 20

def label_connections(gt_graph, inferred_graph, connections, threshold=25):
	g = inferred_graph.clone()
	insert_connections(g, connections)
	edge_index = g.edgeIndex()
	gt_index = gt_graph.edgeIndex()
	for edge in g.edges:
		if hasattr(edge, 'phantom'):
			edge.cost = PHANTOM_BASE + PHANTOM_WEIGHT * edge.segment().length()

			# if it is far from ground truth edge, increase it's edge cost
			'''segment = edge.segment()
			midpoint = segment.start.add(segment.vector().scale(0.5))
			gt_distance = None
			for edge in gt_index.search(midpoint.bounds().add_tol(MAX_GT_DISTANCE)):
				d = edge.segment().distance(midpoint)
				if gt_distance is None or d < gt_distance:
					gt_distance = d
			if gt_distance is None or gt_distance > MAX_GT_DISTANCE:
				edge.cost *= 100'''

	# run dijkstra's on every ground truth road segment whose endpoints match
	# any connections that are traversed in such a search are marked "good"
	def match_point(point):
		best_vertex = None
		best_distance = None
		for edge in edge_index.search(point.bounds().add_tol(threshold)):
			if hasattr(edge, 'phantom'):
				continue
			for vertex in [edge.src, edge.dst]:
				d = point.distance(vertex.point)
				if d < threshold and (best_vertex is None or d < best_distance):
					best_vertex = vertex
					best_distance = d
		return best_vertex

	road_segments, _ =  get_graph_road_segments(gt_graph)
	good_connection_indices = set()
	for rs in road_segments:
		src = match_point(rs.src().point)
		dst = match_point(rs.dst().point)
		if src is None or dst is None:
			continue
		_, edge_path =  shortest_path(src, dst, max_distance=(PHANTOM_WEIGHT+1)*rs.length())
		if edge_path is None:
			continue

		for edge in edge_path:
			if hasattr(edge, 'connection_idx'):
				good_connection_indices.add(edge.connection_idx)

	good_connections = [connection for i, connection in enumerate(connections) if i in good_connection_indices]
	bad_connections = [connection for i, connection in enumerate(connections) if i not in good_connection_indices]
	return good_connections, bad_connections

def visualize_connection(sat, gt_idx, inferred_idx, connection, fname, good=True):
	path = [ Point(p[0], p[1]) for p in connection['path']]
	r = path[0].bounds()
	for p in path:
		r = r.extend(p)
	r = r.add_tol(128)
	r =  Rectangle( Point(0, 0),  Point(sat.shape[0], sat.shape[1])).clip_rect(r)
	im = numpy.copy(sat[r.start.x:r.end.x, r.start.y:r.end.y])
	im_rect =  Rectangle( Point(0, 0),  Point(im.shape[0], im.shape[1]))

	def color_point(p, color, tol=1):
		s = im_rect.clip(p.sub( Point(tol, tol)))
		e = im_rect.clip(p.add( Point(tol, tol)))
		im[s.x:e.x+1, s.y:e.y+1, :] = color

	# draw graph yellow
	for edge in inferred_idx.search(r.add_tol(32)):
		segment = edge.segment()
		start = segment.start.sub(r.start)
		end = segment.end.sub(r.start)
		for p in  draw_line(start, end, r.lengths()):
			color_point(p, [255, 255, 0])

	# draw connection red or green
	for i in xrange(len(path) - 1):
		start = path[i].sub(r.start)
		end = path[i + 1].sub(r.start)
		for p in  draw_line(start, end, r.lengths()):
			if good:
				color_point(p, [0, 255, 0])
			else:
				color_point(p, [255, 0, 0])

	# draw gt graph blue
	for edge in gt_idx.search(r.add_tol(32)):
		segment = edge.segment()
		start = segment.start.sub(r.start)
		end = segment.end.sub(r.start)
		for p in  draw_line(start, end, r.lengths()):
			color_point(p, [0, 0, 255], tol=0)

	Image.fromarray(im.swapaxes(0, 1)).save(fname)

def prepare_connection(sat, outim, inferred_idx, connection, size=320):
	path = [ Point(p[0], p[1]) for p in connection['path']]
	r = path[0].bounds()
	for p in path:
		r = r.extend(p)
	l = r.lengths()
	if l.x > 256 or l.y > 256:
		return
	s =  Point((size - l.x)/2, (size - l.y)/2)
	r =  Rectangle(r.start.sub(s), r.end.add(s))
	r =  Rectangle( Point(0, 0),  Point(sat.shape[0], sat.shape[1])).clip_rect(r)
	l = r.lengths()
	im = numpy.zeros((size, size, 6), dtype='uint8')
	im[0:l.x, 0:l.y, 0:3] = sat[r.start.x:r.end.x, r.start.y:r.end.y, :]
	im[0:l.x, 0:l.y, 5] = outim[r.start.x:r.end.x, r.start.y:r.end.y]

	# draw graph
	for edge in inferred_idx.search(r.add_tol(32)):
		segment = edge.segment()
		start = segment.start.sub(r.start)
		end = segment.end.sub(r.start)
		for p in  draw_line(start, end, r.lengths()):
			im[p.x, p.y, 3] = 255

	# draw connection
	for i in xrange(len(path) - 1):
		start = path[i].sub(r.start)
		end = path[i + 1].sub(r.start)
		for p in  draw_line(start, end, r.lengths()):
			im[p.x, p.y, 4] = 255

	return im

def write_connection(sat, outim, inferred_idx, connection, fname):
	im = prepare_connection(sat, outim, inferred_idx, connection)
	Image.fromarray(im[:, :, 0:3].swapaxes(0, 1)).save(fname + '.sat.png')
	Image.fromarray(im[:, :, 3:6].swapaxes(0, 1)).save(fname + '.con.png')

def sample_points(path, sample_freq=5):
	samples = []
	remaining = 0
	for i in xrange(len(path) - 1):
		cur_point = path[i]
		next_point = path[i + 1]
		l = next_point.distance(cur_point)
		d = 0
		while l - d > remaining:
			d += remaining
			remaining = SAMPLE_FREQ
			samples.append(cur_point.add(next_point.sub(cur_point).scale(d / l)))
		remaining -= l - d
	return samples

def extract_features(im, connection):
	path = [ Point(p[0], p[1]) for p in connection['path']]
	samples = sample_points(path)
	softmax_scores = []
	distance_nonroad = []
	for point in samples:
		softmax_scores.append(im[point.x, point.y])


##Creating original graph

In [ ]:
import scipy.ndimage
import skimage.morphology
import os
from PIL import Image
import math
import numpy
from multiprocessing import Pool
import subprocess


in_fname_path = "/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map"
threshold = 10
out_fname_path = "/content/gdrive/My Drive/graphs"
list_of_files = [f for f in os.listdir(in_fname_path) if os.path.isfile(os.path.join(in_fname_path,f))]
for f in list_of_files:
    in_fname = os.path.join(in_fname_path,f)
    im = scipy.ndimage.imread(in_fname)
    im = numpy.swapaxes(im, 0, 1)
    im = im > threshold

    # apply morphological dilation and thinning
    selem = skimage.morphology.disk(2)
    im = skimage.morphology.binary_dilation(im, selem)
    im = skimage.morphology.thin(im)
    im = im.astype('uint8')

    # extract a graph by placing vertices every THRESHOLD pixels, and at all intersections
    vertices = []
    edges = set()
    def add_edge(src, dst):
        if (src, dst) in edges or (dst, src) in edges:
            return
        elif src == dst:
            return
        edges.add((src, dst))
    point_to_neighbors = {}
    q = []
    while True:
        if len(q) > 0:
            lastid, i, j = q.pop()
            path = [vertices[lastid], (i, j)]
            if im[i, j] == 0:
                continue
            point_to_neighbors[(i, j)].remove(lastid)
            if len(point_to_neighbors[(i, j)]) == 0:
                del point_to_neighbors[(i, j)]
        else:
            w = numpy.where(im > 0)
            if len(w[0]) == 0:
                break
            i, j = w[0][0], w[1][0]
            lastid = len(vertices)
            vertices.append((i, j))
            path = [(i, j)]

        while True:
            im[i, j] = 0
            neighbors = []
            for oi in [-1, 0, 1]:
                for oj in [-1, 0, 1]:
                    ni = i + oi
                    nj = j + oj
                    if ni >= 0 and ni < im.shape[0] and nj >= 0 and nj < im.shape[1] and im[ni, nj] > 0:
                        neighbors.append((ni, nj))
            if len(neighbors) == 1 and (i, j) not in point_to_neighbors:
                ni, nj = neighbors[0]
                path.append((ni, nj))
                i, j = ni, nj
            else:
                if len(path) > 1:
                    path =  rdp(path, 2)
                    if len(path) > 2:
                        for point in path[1:-1]:
                            curid = len(vertices)
                            vertices.append(point)
                            add_edge(lastid, curid)
                            lastid = curid
                    neighbor_count = len(neighbors) + len(point_to_neighbors.get((i, j), []))
                    if neighbor_count == 0 or neighbor_count >= 2:
                        curid = len(vertices)
                        vertices.append(path[-1])
                        add_edge(lastid, curid)
                        lastid = curid
                for ni, nj in neighbors:
                    if (ni, nj) not in point_to_neighbors:
                        point_to_neighbors[(ni, nj)] = set()
                    point_to_neighbors[(ni, nj)].add(lastid)
                    q.append((lastid, ni, nj))
                for neighborid in point_to_neighbors.get((i, j), []):
                    add_edge(neighborid, lastid)
                break
    out_fname = os.path.join(out_fname_path,os.path.splitext(f)[0]+'.graph')
    print "Writting to graph file " + os.path.splitext(f)[0]+'.graph'
    with open(out_fname, 'w') as f:
        for vertex in vertices:
            f.write('{} {}\n'.format(vertex[0], vertex[1]))
        f.write('\n')
        for edge in edges:
            f.write('{} {}\n'.format(edge[0], edge[1]))
            f.write('{} {}\n'.format(edge[1], edge[0]))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


Writting to graph file 23879035_15.graph
Writting to graph file 15778915_15.graph
Writting to graph file 17578975_15.graph
Writting to graph file 22678960_15.graph
Writting to graph file 18028915_15.graph
Writting to graph file 19678630_15.graph
Writting to graph file 10678735_15.graph
Writting to graph file 21629005_15.graph
Writting to graph file 18628750_15.graph
Writting to graph file 22378975_15.graph
Writting to graph file 18178915_15.graph
Writting to graph file 11878660_15.graph
Writting to graph file 10078735_15.graph
Writting to graph file 23129440_15.graph
Writting to graph file 25229215_15.graph
Writting to graph file 26279260_15.graph
Writting to graph file 23128870_15.graph
Writting to graph file 15928900_15.graph
Writting to graph file 22829485_15.graph
Writting to graph file 20728975_15.graph
Writting to graph file 25529260_15.graph
Writting to graph file 18328810_15.graph
Writting to graph file 23428915_15.graph
Writting to graph file 21628990_15.graph
Writting to grap

##Generating training example for connection classifier
Use the segmentation output and inferred graphs from the training set to generate training examples for the connection classifier:

In [ ]:
!mkdir '/content/gdrive/My Drive/examples3'
!mkdir '/content/gdrive/My Drive/examples3/good'
!mkdir '/content/gdrive/My Drive/examples3/bad'

In [ ]:
import scipy.ndimage
from PIL import Image
import numpy
from multiprocessing import Pool
import subprocess
import os


	
sat_path = "/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/sat_resized" #1024
graph_path = "/content/gdrive/My Drive/graphs"  
out_im_path = "/content/gdrive/My Drive/output/img" #1024
out_graph_path = "/content/gdrive/My Drive/output/graph" 
dst_path = "/content/gdrive/My Drive/examples3"

regions = [fname.split('.tif')[0] for fname in os.listdir(out_im_path) if '.tif' in fname]



In [ ]:
#5.50 - 12.48 Took 7hours
def process(region):
	print region
	city = region.split('_')[0]
	#offset_x, offset_y = int(region.split('_')[1]), int(region.split('_')[2])
	offset_x, offset_y =0,0
	sat = scipy.ndimage.imread('{}/sat_resized{}_resized.tif'.format(sat_path, region))
	sat = sat.swapaxes(0, 1)
	im = scipy.ndimage.imread('{}/{}.tif'.format(out_im_path, region)).astype('float32') / 255.0
	im = im.swapaxes(0, 1)
	g =  read_graph('{}/{}.graph'.format(out_graph_path, region))
	g_idx = g.edgeIndex()

	gt =  read_graph('{}/{}_15.graph'.format(graph_path, city))
	offset =  Point(offset_x * 1024, offset_y * 1024)
	for vertex in gt.vertices:
		vertex.point = vertex.point.sub(offset)

	gt_idx = gt.edgeIndex()

	connections =  get_connections(g, im, limit=512)
	good, bad = label_connections(gt, g, connections)
	for i, connection in enumerate(good):
		write_connection(sat, im, g_idx, connection, '{}/good/{}_{}'.format(dst_path, region, i))
	for i, connection in enumerate(bad):
		write_connection(sat, im, g_idx, connection, '{}/bad/{}_{}'.format(dst_path, region, i))
p = Pool()
p.map(process, regions)
p.close()

10978660_15


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  import sys


18178975_15


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
  if __name__ == '__main__':


18328720_15
10978645_15
18328765_15
10828900_15
10828870_15
18328795_15
10828855_15
10828825_15
10828810_15
18328810_15
18328885_15
18328900_15
10828795_15
10828780_15
18328915_15
10828765_15
18328930_15
10828750_15
18328945_15
10828735_15
18328975_15
18478720_15
10828705_15
10828690_15
18478750_15
18478765_15
18478780_15
18478795_15
10828675_15
18478885_15
18478915_15
10828660_15
10828645_15
18478960_15
10678885_15
10678870_15
10678825_15
10678810_15
10678795_15
10678780_15
18478975_15
18628720_15
18628735_15
10678765_15
18628750_15
10678750_15
18628765_15
18628780_15
10678735_15
18628795_15
18628885_15
18628900_15
10678720_15
18628915_15
10678705_15
18628930_15
18778720_15
18778735_15
10678690_15
18778750_15
18778765_15
10678675_15
18778780_15
10678660_15
10528795_15
18778795_15
18928720_15
10528780_15
18928735_15
10528765_15
18928750_15
18928765_15
10528750_15
18928780_15
18928795_15
19078735_15
19078750_15
10528735_15
19078765_15
19228735_15
19228750_15
19528630_15
19528645_15
1052

##Train connection classifier

In [ ]:

import os
os.chdir("/content/gdrive/My Drive/examples3/good")



!ls -lR *sat.png | wc -l
!ls -lR *con.png | wc -l

4286
4350


In [ ]:
#2.00 
import sys
sys.path.append("/content/gdrive/My Drive")
import model_connect as model

import numpy
import os
from PIL import Image
import random
import scipy.ndimage
import subprocess
import sys
import tensorflow as tf
import time
import os.path


data_path = '/content/gdrive/My Drive/examples3'
model_path = '/content/gdrive/My Drive/cmodel'

print 'loading tiles'

def load_ims(path, limit=None):
    fnames = [path + '/' + fname for fname in os.listdir(path) if '.sat.png' in fname]
    if limit is not None and len(fnames) > limit:
    	fnames = random.sample(fnames, limit)
    ims = []
    for sat_fname in fnames:
    	con_fname = sat_fname.replace('.sat.', '.con.')
        if (os.path.isfile(con_fname))== False :
            continue
        im = numpy.zeros((320, 320, 6), dtype='uint8')
        im[:, :, 0:3] = scipy.ndimage.imread(sat_fname).swapaxes(0, 1)
        im[:, :, 3:6] = scipy.ndimage.imread(con_fname).swapaxes(0, 1)
        ims.append(im)
    return ims

def extract_example(tile):
	im, label = tile
	i = random.randint(0, 64)
	j = random.randint(0, 64)
	return im[i:i+256, j:j+256, :], label

good_tiles = [(im, 1) for im in load_ims('{}/good'.format(data_path),limit=None)]
bad_tiles = [(im, 0) for im in load_ims('{}/bad'.format(data_path), limit=len(good_tiles))]
random.shuffle(good_tiles)
random.shuffle(bad_tiles)
val_tiles = good_tiles[0:512] + bad_tiles[0:512]
train_tiles = good_tiles[512:] + bad_tiles[512:]

print 'using {} train tiles, {} val tiles, {} good tiles, {} bad tiles'.format(len(train_tiles), len(val_tiles), len(good_tiles), len(bad_tiles))

latest_path = model_path + '/model_latest/model'
best_path = model_path + '/model_best/model'

m = model.Model()
session = tf.Session()
session.run(m.init_op)

best_accuracy = None

def epoch_to_learning_rate(epoch):
	if epoch < 100:
		return 1e-4
	elif epoch < 200:
		return 1e-5
	elif epoch < 300:
		return 1e-6
	else:
		return 1e-7

for epoch in xrange(10):
	start_time = time.time()
	train_losses = []
	for _ in xrange(8192):
		tiles = random.sample(train_tiles, model.BATCH_SIZE)
		examples = [extract_example(tile) for tile in tiles]
		_, loss = session.run([m.optimizer, m.loss], feed_dict={
			m.is_training: True,
			m.inputs: [example[0].astype('float32') / 255 for example in examples],
			m.targets: [[example[1]] for example in examples],
			m.learning_rate: epoch_to_learning_rate(epoch),
		})
		train_losses.append(loss)
	train_loss = numpy.mean(train_losses)
	train_time = time.time()

	val_losses = []
	val_accuracies = []
	for i in xrange(0, len(val_tiles), model.BATCH_SIZE):
		examples = [extract_example(tile) for tile in val_tiles[i:i+model.BATCH_SIZE]]
		loss, outputs = session.run([m.loss, m.outputs], feed_dict={
			m.is_training: False,
			m.inputs: [example[0].astype('float32') / 255 for example in examples],
			m.targets: [[example[1]] for example in examples],
		})
		val_losses.append(loss)
		for j in xrange(len(examples)):
			output = outputs[j] > 0.5
			gt = examples[j][1] > 0.5
			if output == gt:
				val_accuracies.append(1.0)
			else:
				val_accuracies.append(0.0)

	val_loss = numpy.mean(val_losses)
	val_accuracy = numpy.mean(val_accuracies)
	val_time = time.time()

	print 'iteration {}: train_time={}, val_time={}, train_loss={}, val_loss={}, val_acc={}/{}'.format(epoch, int(train_time - start_time), int(val_time - train_time), train_loss, val_loss, val_accuracy, best_accuracy)

	m.saver.save(session, latest_path)
	if best_accuracy is None or val_accuracy > best_accuracy:
		best_accuracy = val_accuracy
		m.saver.save(session, best_path)

loading tiles


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


using 429 train tiles, 647 val tiles, 941 good tiles, 135 bad tiles


NameError: ignored

In [ ]:
print len(os.listdir('/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/sat'))
print len(os.listdir('/content/gdrive/My Drive/graphs'))
print len(os.listdir('/content/gdrive/My Drive/output/graph'))
print len(os.listdir('/content/gdrive/My Drive/output/img'))

print len(os.listdir('/content/gdrive/My Drive/examples/bad'))
print len(os.listdir('/content/gdrive/My Drive/examples/good'))

print len(os.listdir('/content/gdrive/My Drive/examples3/bad'))
print len(os.listdir('/content/gdrive/My Drive/examples3/good'))

print len(os.listdir('/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/sat_resized'))
print len(os.listdir('/content/gdrive/My Drive/mass_roads/train/map_resized'))


1108
1108
1108
1108
88
52


OSError: ignored

In [ ]:
import cv2
import numpy

img = cv2.imread('/content/gdrive/My Drive/output/graph/10078660_15.graph')
img2 = cv2.imread('/content/gdrive/My Drive/output/img/10078660_15.tif')

img2.shape


(1024, 1024, 3)

In [ ]:
img3 = cv2.imread('/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map/10078660_15.tif')
img3.shape

(1500, 1500, 3)

##Resizing trainning images to 1024x1024

In [ ]:
from PIL import Image
import os, sys

def resizeImage(infile, output_dir="", size=(1024,1024)):
  outfile = os.path.splitext(infile)[0]+"_resized"
  extension = os.path.splitext(infile)[1]
    
  
  if infile != outfile:
    try :
      im = Image.open(infile)
      #print "outfile opened"
      im.thumbnail(size, Image.ANTIALIAS)
      im.save(output_dir+outfile+extension)
      print "saved", outfile
    except IOError as e:
      print e, infile



In [ ]:
!mkdir '/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map_resized'

In [ ]:
from os import path
os.chdir('/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map')

In [ ]:
!pwd

/content/gdrive/My Drive/mass_roads/train/map


In [ ]:
output_dir = '/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map_resized/'
dir = '/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/map'

for file in os.listdir(dir):
  
  resizeImage( file,output_dir)

saved 23879035_15_resized
saved 15778915_15_resized
saved 17578975_15_resized
saved 22678960_15_resized
saved 18028915_15_resized
saved 19678630_15_resized
saved 10678735_15_resized
saved 21629005_15_resized
saved 18628750_15_resized
saved 22378975_15_resized
saved 18178915_15_resized
saved 11878660_15_resized
saved 10078735_15_resized
saved 23129440_15_resized
saved 25229215_15_resized
saved 26279260_15_resized
saved 23128870_15_resized
saved 15928900_15_resized
saved 22829485_15_resized
saved 20728975_15_resized
saved 25529260_15_resized
saved 18328810_15_resized
saved 23428915_15_resized
saved 21628990_15_resized
saved 10978945_15_resized
saved 12028735_15_resized
saved 26728780_15_resized
saved 26429245_15_resized
saved 26578705_15_resized
saved 22679050_15_resized
saved 24778855_15_resized
saved 22679485_15_resized
saved 25529230_15_resized
saved 24779245_15_resized
saved 17728930_15_resized
saved 15928915_15_resized
saved 27028690_15_resized
saved 17428750_15_resized
saved 214788

In [ ]:
import os
import shutil
sourcepath='/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/'
sourcefiles = os.listdir(sourcepath)
destinationpath = '/content/gdrive/My Drive/Final Year Project Data/mass_roads/train/sat_resized'
for file in sourcefiles:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sourcepath,file), os.path.join(destinationpath,file))